In [12]:
import sqlite3

conn = sqlite3.connect("kayak.db")


In [13]:
cursor = conn.cursor()

cursor.executescript("""
CREATE TABLE IF NOT EXISTS cities (
    city_id INTEGER PRIMARY KEY,
    city_name TEXT,
    latitude REAL,
    longitude REAL
);

CREATE TABLE IF NOT EXISTS weather_daily (
    weather_id INTEGER,
    city_id INTEGER,
    forecast_date DATE,
    temp_mean_c REAL,
    temp_min_c REAL,
    temp_max_c REAL,
    pop_mean REAL,
    rain_mm_sum REAL,
    FOREIGN KEY (city_id) REFERENCES cities(city_id)
);

CREATE TABLE IF NOT EXISTS hotels (
    hotel_id INTEGER PRIMARY KEY,
    city_id INTEGER,
    hotel_name TEXT,
    url TEXT,
    latitude REAL,
    longitude REAL,
    rating REAL,
    description TEXT,
    FOREIGN KEY (city_id) REFERENCES cities(city_id)
);
""")

conn.commit()


In [14]:
import pandas as pd


In [19]:
query = """
SELECT
    c.city_name,
    c.latitude,
    c.longitude,
    AVG(w.temp_mean_c) AS avg_temperature,
    AVG(w.pop_mean) AS mean_pop
FROM cities c
JOIN weather_daily w
    ON c.city_id = w.city_id
GROUP BY c.city_name, c.latitude, c.longitude;
"""

df_sql = pd.read_sql(query, conn)
df_sql


,city_name,latitude,longitude,avg_temperature,mean_pop
0,Aigues Mortes,43.566152,4.191540,16.658236,0.000000
1,Aix en Provence,43.529842,5.447474,17.334917,0.029375
2,Amiens,49.894171,2.295695,12.060292,0.000000
3,Annecy,45.899235,6.128885,10.527569,0.000000
4,Ariege,42.945537,1.406554,13.122486,0.179444
5,Avignon,43.949249,4.805901,15.414681,0.000000
6,Bayeux,49.276462,-0.702474,12.503000,0.019444
7,Bayonne,43.494514,-1.473666,16.684833,0.000000
8,Besancon,47.238022,6.024362,10.013250,0.000000
9,Biarritz,43.483252,-1.559278,17.002458,0.000000


In [20]:
import plotly.express as px

fig = px.scatter_mapbox(
    df_sql,
    lat="latitude",
    lon="longitude",
    color="avg_temperature",
    size="avg_temperature",
    hover_name="city_name",
    color_continuous_scale="Bluered",
    zoom=4.5,
    height=650,
    title="Top destinations selon les conditions météorologiques"
)

fig.update_layout(mapbox_style="carto-positron")
fig.show()


In [27]:
query_top5 = """
SELECT
    c.city_name,
    c.latitude,
    c.longitude,
    AVG(w.temp_mean_c) AS avg_temperature,
    AVG(w.pop_mean) AS mean_pop
FROM cities c
JOIN weather_daily w ON c.city_id = w.city_id
GROUP BY c.city_name, c.latitude, c.longitude
HAVING AVG(w.temp_mean_c) BETWEEN 17 AND 22
ORDER BY AVG(w.pop_mean) ASC,
         AVG(w.temp_mean_c) DESC,
         c.city_name ASC
LIMIT 5;
"""
df_top5 = pd.read_sql(query_top5, conn)
df_top5

,city_name,latitude,longitude,avg_temperature,mean_pop
0,Toulouse,43.604464,1.444243,17.544708,0.000000
1,Biarritz,43.483252,-1.559278,17.002458,0.000000
2,Carcassonne,43.213036,2.349107,17.150194,0.003333
3,Aix en Provence,43.529842,5.447474,17.334917,0.029375
4,Collioure,42.525050,3.083155,17.329778,0.049306


In [34]:
import plotly.express as px

fig = px.scatter_mapbox(
    df_top5,
    lat="latitude",
    lon="longitude",
    hover_name="city_name",
    hover_data={
        "avg_temperature": ":.1f",
        "mean_pop": ":.3f",
        "latitude": False,
        "longitude": False,
    },
    color="city_name",        
    zoom=4,
    height=650,
    title="Top 5 villes sélectionnées selon la météo"
)

fig.update_layout(
    mapbox_style="open-street-map",
    margin=dict(l=10, r=10, t=60, b=10),
    legend_title_text="Ville"
)
fig.update_traces(marker=dict(size=14))

fig.show()

In [35]:
query_hotels = """
SELECT
    h.hotel_id,
    h.hotel_name,
    h.latitude,
    h.longitude,
    h.rating,
    h.url,
    c.city_name
FROM hotels h
JOIN cities c
    ON h.city_id = c.city_id
WHERE h.latitude IS NOT NULL
  AND h.longitude IS NOT NULL;
"""
df_hotels = pd.read_sql(query_hotels, conn)
df_hotels.head()


,hotel_id,hotel_name,latitude,longitude,rating,url,city_name
0,1,CABADOL - Appartement Aix centre historique - ...,43.525963,5.451841,9.4,https://www.booking.com/hotel/fr/appartement-a...,Aix en Provence
1,2,Chambre style balinais,43.528428,5.452377,9.4,https://www.booking.com/hotel/fr/chambre-style...,Aix en Provence
2,3,Château de Saint-Girons - Luxury Guest House i...,43.498920,5.382602,9.8,https://www.booking.com/hotel/fr/chateau-de-sa...,Aix en Provence
3,4,Domaine de Saint Clair,43.499156,5.432056,9.7,https://www.booking.com/hotel/fr/domaine-de-sa...,Aix en Provence
4,5,Duplex indépendant avec clim et parking St Donat,43.560699,5.451552,9.8,https://www.booking.com/hotel/fr/independent-a...,Aix en Provence


In [36]:
df_top = df_hotels.sort_values("rating", ascending=False).head(200)

fig = px.scatter_mapbox(
    df_top,
    lat="latitude",
    lon="longitude",
    color="rating",
    hover_name="hotel_name",
    hover_data={"city_name": True, "rating": True, "url": True},
    zoom=4.5,
    height=650,
    title="Top 20 Hôtels par note"
)
fig.update_layout(mapbox_style="carto-positron")
fig.show()
